In [2]:
import pandas as pd
import json
import re
import os
import requests
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
passivator_df = pd.read_csv("data/final_extraction/df_has_passivator.csv")
passivator_df

,Unnamed: 0,perovskite_composition,electron_transport_layer,pin_nip_structure,hole_transport_layer,passivating_molecule,control_pce,control_voc,treated_pce,treated_voc,...,treated_eqe,treated_stabilized_pce,control_fill_factor,treated_fill_factor,controljsc,treatedjsc,treated_hysteresis_index,binding_energy,fill_factor,treated_iscc
0,0,MAPbI3,PEDOT:PSS,NIP,PCBM,DAAB,4.50,0.870,12.60,0.920,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,formamidine sulfinic acid (FSA),NaN,NaN,27.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Cs 0.05 FA 0.9 MA 0.05 PbI 3,NaN,NaN,NaN,Cyanoguanidine diiodide,20.44,1077.000,23.04,1119.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Cs 0.17 FA 0.83 Pb(I 0.9 Br 0.1 ) 3 (WBg),TiO2,NaN,Spiro-OMeTAD,tBBAI,14.91,1.070,23.50,1.140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,FAPbI3 0.85 (-MAPbBr3) 0.15,Spiro-OMeTAD,NIP,NaN,7-amino-4-trifluoromethylcoumarin (C151),20.44,1.090,22.46,1.140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001,2532,MAPbI3,NaN,NaN,NaN,OTTM,18.45,1.080,21.01,1.120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002,2533,MAPbI3,PCBM,PIN,NiOx,PbS,15.26,1.002,18.74,1.025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003,2534,"Cu2ZnSn(S,Se)4 (C:SSe)",NaN,NaN,NaN,Vanadium (V),8.28,0.454,11.11,0.458,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004,2536,NaN,TiO2,NaN,Spiro-MeOTAD,K-NAA,19.86,1.140,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
passivator_conversions_path = "data/passivator_conversions.json"
with open(passivator_conversions_path, "r", encoding="utf-8") as f:
    passivator_conversions = json.load(f)

In [100]:
def get_IUPAC_name(passivator):
    if passivator in passivator_conversions:
        value = passivator_conversions[passivator]
        if value is None:
            return None
        if "," in value:
            return value.split(",")[0]
        if "/" in value:
            return value.split("/")[0]
        return passivator_conversions[passivator]
    else:
        return "unknown"

In [101]:
passivator_df["IUPAC_passivator"] = passivator_df["passivating_molecule"].apply(get_IUPAC_name)
passivator_df

,Unnamed: 0,perovskite_composition,electron_transport_layer,pin_nip_structure,hole_transport_layer,passivating_molecule,control_pce,control_voc,treated_pce,treated_voc,...,treated_stabilized_pce,control_fill_factor,treated_fill_factor,controljsc,treatedjsc,treated_hysteresis_index,binding_energy,fill_factor,treated_iscc,IUPAC_passivator
0,0,MAPbI3,PEDOT:PSS,NIP,PCBM,DAAB,4.50,0.870,12.60,0.920,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,diammonium azobenzene
1,1,NaN,NaN,NaN,NaN,formamidine sulfinic acid (FSA),NaN,NaN,27.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,3,Cs 0.05 FA 0.9 MA 0.05 PbI 3,NaN,NaN,NaN,Cyanoguanidine diiodide,20.44,1077.000,23.04,1119.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cyanoguanidine diiodide
3,4,Cs 0.17 FA 0.83 Pb(I 0.9 Br 0.1 ) 3 (WBg),TiO2,NaN,Spiro-OMeTAD,tBBAI,14.91,1.070,23.50,1.140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,5,FAPbI3 0.85 (-MAPbBr3) 0.15,Spiro-OMeTAD,NIP,NaN,7-amino-4-trifluoromethylcoumarin (C151),20.44,1.090,22.46,1.140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001,2532,MAPbI3,NaN,NaN,NaN,OTTM,18.45,1.080,21.01,1.120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2002,2533,MAPbI3,PCBM,PIN,NiOx,PbS,15.26,1.002,18.74,1.025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lead(II) sulfide
2003,2534,"Cu2ZnSn(S,Se)4 (C:SSe)",NaN,NaN,NaN,Vanadium (V),8.28,0.454,11.11,0.458,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vanadium
2004,2536,NaN,TiO2,NaN,Spiro-MeOTAD,K-NAA,19.86,1.140,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [103]:
def get_smiles(row):
    molecule_name = row["IUPAC_passivator"]
    if molecule_name is None:
        molecule_name = row["passivating_molecule"]
    base_url = "https://opsin.ch.cam.ac.uk/opsin/"
    smiles_url = base_url + molecule_name + ".smi"
    r = requests.get(smiles_url)
    return r.text if r.status_code == 200 else None

In [104]:
passivator_df["passivator_smiles"] = passivator_df.apply(get_smiles, axis=1)

In [107]:
smiles_df = passivator_df[passivator_df["passivator_smiles"].isna() == False]
smiles_df.to_csv('data/final_extraction/data_with_smiles.csv', index=False)

In [ ]:
None_passivators = passivator_df[passivator_df["IUPAC_passivator"].isna()]
passivators_list = None_passivators["passivating_molecule"].tolist()

In [76]:
passivator_df.to_csv('data/final_extraction/iupac_df.csv', index=False)

In [ ]:
iupac_df = passivator_df[passivator_df["IUPAC_passivator"].isna() == False]

<class 'pandas.core.frame.DataFrame'>
Index: 1034 entries, 0 to 2003
Data columns (total 37 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                1034 non-null   int64  
 1   perovskite_composition    391 non-null    object 
 2   electron_transport_layer  374 non-null    object 
 3   pin_nip_structure         277 non-null    object 
 4   hole_transport_layer      312 non-null    object 
 5   passivating_molecule      1034 non-null   object 
 6   control_pce               548 non-null    float64
 7   control_voc               444 non-null    float64
 8   treated_pce               763 non-null    float64
 9   treated_voc               541 non-null    float64
 10  test_1                    960 non-null    object 
 11  test_2                    276 non-null    object 
 12  structure_pin_nip         9 non-null      object 
 13  treated_jsc               17 non-null     float64
 14  treated_ff   

In [62]:
passivators_list

['formamidine sulfinic acid (FSA)',
 'tBBAI',
 '7-amino-4-trifluoromethylcoumarin (C151)',
 'ICBA',
 'MASCN and FASCN',
 'DPDC',
 'Chen et al. introduced Rb2CO3 layer at the SnO2/perovskite interface',
 'N-Ti 3 C 2',
 'MeO-2PACz',
 'SPM/PEAI',
 'N2200',
 'BDAI2',
 'poly(2-(2-methoxyethoxy)ethylmethacrylate)',
 'DFH',
 'S-benzyl-L-cysteine',
 'DTPA-CN',
 'Molybdenum-sulfur-containing precursor',
 'Cs0.05MA0.15FA0.8Pb(I1-xBr_x)_3',
 'Siloxane-based self-assembled molecule with nitrile group',
 'Potassium laurate (KLA)',
 'KCFSO',
 '(5-AVA)x(MA)1-x PbI3',
 'TPPP(Cl)',
 '1-iodobutane, 1-phenylethylamine, PbI2',
 '2-ThMAI',
 'EDAI2 and GlyHCl',
 'polymerized-fluorocarbon coating (PPFC)',
 'MPA-TB-CA',
 'MBene',
 'Lead carbanion complex',
 'MBene',
 'EDABr2',
 'PbX2 (halide ligands)',
 '2-ADAHCl',
 'C60, SnO2, NiOx, and self-assembled molecules',
 'DB24C8 and HABr',
 'B-2D passivation (ACI and RP phases)',
 'poly(N-isopropylacrylamide) microgels (MGs)',
 'DPEPO',
 'CzCy',
 'NiNH2OEP',
 'YZ-3

In [30]:
def get_unconverted_passivators(passivators_list):
    passivators = set()
    for item in passivators_list:
        passivators.add(item)
    passivators = [passivator for passivator in passivators if passivator not in passivator_conversions and passivator not in [None, 'none']]
    return passivators    
        

In [ ]:
PREFIX = """
Role:
You are a helpful scientific assistant specializing in passivating molecules for perovskite solar cells.

Task:
Given a list of molecule names, provide the full name of each molecule in IUPAC nomenclature. The IUPAC name must be parseable into SMILES format.

Rules:
-If the molecule name is already in IUPAC format, return it as is.
-If there are multiple molecule names given, just use the first one.
-If the name contains additional descriptive words (e.g., "passivating," "functionalized"), extract only the molecule name and convert it to IUPAC format.
-If the full molecule name cannot be parsed into SMILES after fully reasoning through it multiple times, return None.
-Ensure no molecules are left out.

Output Format:
Provide a JSON object where each key is the provided molecule name and the value is the corresponding IUPAC name or None.

json
Copy
{
    "molecule_name_1": "IUPAC_name_or_None",
    "molecule_name_2": "IUPAC_name_or_None"
}
Important Notes:

-Focus only on the parts of the string that represent the molecule name.

-Double-check that the IUPAC name can be parsed into SMILES. If not, return None.
-If the molecule is not relevant to passivating perovskite solar cells, return None.

Example Input:

{
    "ethylammonium bromide": "ethylammonium bromide",
    "passivating 2-phenylethylamine": "2-phenylethylamine",
    "CF3PEAI": "2-(4-(Trifluoromethyl)phenyl)ethylammonium iodide",
    "2D perovskite": None
}
Begin converting!
"""

In [79]:
def get_new_passivator_conversions(passivators, api_key_name="OPENAI_API_KEY", base_url="https://api.openai.com/v1/", model_name="gpt-4o"):
    api_key = os.getenv(api_key_name)
    client = OpenAI(api_key=api_key, base_url=base_url)
    system_content = PREFIX
    i = 0
    user_content = str(passivators)

    messages = [
        {
            "role": "system",
            "content": system_content
        },
        {
            "role": "user",
            "content": user_content
        }
    ]
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        stream=False
    )
    output = response.choices[0].message.content
    json_match = re.search(r"\{.*\}", output, re.DOTALL)

    if json_match:
        raw_json = json_match.group(0).strip()
    else:
        print("No JSON found")
        return {}
    try:
        parsed_data = json.loads(raw_json)
        return parsed_data
    except json.JSONDecodeError as e:
        print("Error creating JSON", e)
        return {}

In [34]:
passivators_list = passivator_df["passivating_molecule"].tolist()

In [80]:
len(passivators_list)

972

In [37]:
unconverted_passivators = get_unconverted_passivators(passivators_list)

In [39]:
len(unconverted_passivators)

329

In [81]:
new_passivators = get_new_passivator_conversions(passivators_list)

In [82]:
new_passivators

{'formamidine sulfinic acid (FSA)': 'formamidine sulfinic acid',
 'tBBAI': None,
 '7-amino-4-trifluoromethylcoumarin (C151)': '7-amino-4-(trifluoromethyl)chromen-2-one',
 'ICBA': None,
 'MASCN and FASCN': None,
 'DPDC': None,
 'Chen et al. introduced Rb2CO3 layer at the SnO2/perovskite interface': None,
 'N-Ti 3 C 2': None,
 'MeO-2PACz': None,
 'SPM/PEAI': None,
 'N2200': None,
 'BDAI2': None,
 'poly(2-(2-methoxyethoxy)ethylmethacrylate)': 'poly(2-(2-methoxyethoxy)ethyl methacrylate)',
 'DFH': None,
 'S-benzyl-L-cysteine': 'S-benzyl-L-cysteine',
 'DTPA-CN': None,
 'Molybdenum-sulfur-containing precursor': None,
 'Cs0.05MA0.15FA0.8Pb(I1-xBr_x)_3': None,
 'Siloxane-based self-assembled molecule with nitrile group': None,
 'Potassium laurate (KLA)': 'potassium dodecanoate',
 'KCFSO': None,
 '(5-AVA)x(MA)1-x PbI3': None,
 'TPPP(Cl)': None,
 '1-iodobutane, 1-phenylethylamine, PbI2': None,
 '2-ThMAI': None,
 'EDAI2 and GlyHCl': None,
 'polymerized-fluorocarbon coating (PPFC)': None,
 'MPA-TB

In [ ]:
### second pass to ensure not missing anything (GPT-4o takes less time for this)
None_passivators = [k for k, v in new_passivators.items() if v == None or v == "None"]
extracted_passivators = {k: v for k, v in new_passivators.items() if v != None or v != "None"}

None_passivators = passivators_list
second_pass = get_new_passivator_conversions(None_passivators)

In [84]:
second_pass

{'formamidine sulfinic acid (FSA)': 'formamidine sulfinic acid',
 'tBBAI': 'tert-butyl benzoate ammonium iodide',
 '7-amino-4-trifluoromethylcoumarin (C151)': '7-amino-4-(trifluoromethyl)chromen-2-one',
 'ICBA': None,
 'MASCN and FASCN': None}

In [85]:
extracted_passivators.update(second_pass)

In [88]:
new_vals = {
  "formamidine sulfinic acid (FSA)": "formamidine sulfinic acid",
  "tBBAI": "tert-butylammonium iodide",
  "7-amino-4-trifluoromethylcoumarin (C151)": "7-amino-4-(trifluoromethyl)chromen-2-one",
  "ICBA": "indene-C60 bisadduct",
  "MASCN and FASCN": "methylammonium thiocyanate",
  "DPDC": "diphenyldithiocarbamate",
  "Chen et al. introduced Rb2CO3 layer at the SnO2/perovskite interface": None,
  "N-Ti 3 C 2": None,
  "MeO-2PACz": None,
  "SPM/PEAI": None,
  "N2200": None,
  "BDAI2": "benzylammonium iodide",
  "poly(2-(2-methoxyethoxy)ethylmethacrylate)": "poly(2-(2-methoxyethoxy)ethyl methacrylate)",
  "DFH": None,
  "S-benzyl-L-cysteine": "S-benzyl-L-cysteine",
  "DTPA-CN": None,
  "Molybdenum-sulfur-containing precursor": None,
  "Cs0.05MA0.15FA0.8Pb(I1-xBr_x)_3": None,
  "Siloxane-based self-assembled molecule with nitrile group": None,
  "Potassium laurate (KLA)": "potassium dodecanoate",
  "KCFSO": None,
  "(5-AVA)x(MA)1-x PbI3": None,
  "TPPP(Cl)": None,
  "1-iodobutane, 1-phenylethylamine, PbI2": "1-iodobutane",
  "2-ThMAI": None,
  "EDAI2 and GlyHCl": None,
  "polymerized-fluorocarbon coating (PPFC)": None,
  "MPA-TB-CA": None,
  "MBene": None,
  "Lead carbanion complex": None,
  "EDABr2": None,
  "PbX2 (halide ligands)": None,
  "2-ADAHCl": None,
  "C60, SnO2, NiOx, and self-assembled molecules": "buckminsterfullerene",
  "DB24C8 and HABr": None,
  "B-2D passivation (ACI and RP phases)": None,
  "poly(N-isopropylacrylamide) microgels (MGs)": "poly(N-isopropylacrylamide)",
  "DPEPO": None,
  "CzCy": None,
  "NiNH2OEP": None,
  "YZ-302": None,
  "Dual-additive strategy using 1-aminopyridinium iodide (PyNI) and 5-aminovaleric acid (5AVA)": None,
  "P-(NDI2OD-T2)": None,
  "PTAA": "poly(triarylamine)",
  "CdS": "cadmium sulfide",
  "CPT": None,
  "SnF2, SnCl2, SnI2, piperazine, hypophosphorous acid (HPA), cobaltocene, 2-aminopyrazine (APZ), N,N-methylenebis(acrylamide) (MBAA), phthalimide (PTM), thiourea-based additives, carbazole (CBZ), dimethyl ketoxime (DMKO)": "tin(II) fluoride",
  "D-3PACz": None,
  "TMTA": None,
  "PDF-co-BBPB": None,
  "edamine": None,
  "B-SubPc-F": None,
  "PM-syn and TRX-syn": None,
  "mFPEAI and MeCZEAI": None,
  "F-PEAI": None,
  "Pt QDs": None,
  "Silicon nitride (SiN_x)": "silicon nitride",
  "OAc/OAm": None,
  "D18-Cl": None,
  "BMIMBF4": None,
  "6TIC-4F": None,
  "PbI2 and MACl": "lead(II) iodide",
  "Oleylamine Iodide (OAmI) and 4-Fluorobenzylamine Hydroiodide (4-F-PMAI)": "oleylamine iodide",
  "X22 and X23": None,
  "Cl-BSM": None,
  "MASnF2Cl + MASnI3 +4-Bromophenyl acrylate": None,
  "p(HEMA-co-DEAMA)": "poly(2-hydroxyethyl methacrylate-co-diethyl aminoethyl methacrylate)",
  "Copper-porphyrin complex": None,
  "[CYS][PbCl2]": None,
  "BA2MA3Pb4I13": None,
  "Oleylamine and Oleic Acid": "oleylamine",
  "CsAc and MAAc": None,
  "Graphene": "graphene",
  "2-pyrrolidin-1-ium-1-ethylammonium tetrafluoroborate (PyE(BF4)2)": "2-pyrrolidinium-1-ethylammonium tetrafluoroborate",
  "PBI-SeO10": None,
  "2PACz": None,
  "PMMA with titania nanorod arrays": None,
  "Zn-PP": None,
  "1H6An": None,
  "2-MP": None,
  "Cu-MOF": None,
  "DPIPT": None,
  "ACP": None,
  "PCN": None,
  "Cd-MOF": None,
  "1,4-bis(trimethylsilyl)-2,3,5,6-tetramethyl-1,4-dihydropyrazine (TM-DHP)": "1,4-bis(trimethylsilyl)-2,3,5,6-tetramethylpyrazin-5-ium",
  "SnF2, FSA, HPA": "tin(II) fluoride",
  "NMABr+Al2O3": None,
  "4-A, 4-C, Phen": None,
  "FAMAPbI3-xBrx": None,
  "BDTOxide-MTP": None,
  "[AOEMIM][BF4]": None,
  "1,3,3-trimethylindolino-8′-methoxybenzopyrylospiran (OMe-SP)": "1,3,3-trimethylindolino-8-methoxybenzo[b]pyran",
  "C9-thione (FN-S)": None,
  "Native ligand (combination of carboxylic acids and amines)": None,
  "Li-TFSI": None
}

In [89]:
for key in new_vals:
    if key in extracted_passivators:
        extracted_passivators[key] = new_vals[key]

In [96]:
{k: v for k,v in extracted_passivators.items() if v == None}

{'Chen et al. introduced Rb2CO3 layer at the SnO2/perovskite interface': None,
 'N-Ti 3 C 2': None,
 'MeO-2PACz': None,
 'SPM/PEAI': None,
 'N2200': None,
 'DFH': None,
 'DTPA-CN': None,
 'Molybdenum-sulfur-containing precursor': None,
 'Cs0.05MA0.15FA0.8Pb(I1-xBr_x)_3': None,
 'Siloxane-based self-assembled molecule with nitrile group': None,
 'KCFSO': None,
 '(5-AVA)x(MA)1-x PbI3': None,
 'TPPP(Cl)': None,
 '2-ThMAI': None,
 'EDAI2 and GlyHCl': None,
 'polymerized-fluorocarbon coating (PPFC)': None,
 'MPA-TB-CA': None,
 'MBene': None,
 'Lead carbanion complex': None,
 'EDABr2': None,
 'PbX2 (halide ligands)': None,
 '2-ADAHCl': None,
 'DB24C8 and HABr': None,
 'B-2D passivation (ACI and RP phases)': None,
 'DPEPO': None,
 'CzCy': None,
 'NiNH2OEP': None,
 'YZ-302': None,
 'Dual-additive strategy using 1-aminopyridinium iodide (PyNI) and 5-aminovaleric acid (5AVA)': None,
 'P-(NDI2OD-T2)': None,
 'CPT': None,
 'D-3PACz': None,
 'TMTA': None,
 'PDF-co-BBPB': None,
 'edamine': None,
 'B

In [45]:
### cleaning any multiples:
for key in extracted_passivators:
    value = extracted_passivators[key]
    if isinstance(value, dict):
        extracted_passivators[key] = value[list(value.keys())[0]]

In [46]:
extracted_passivators

{'2,5-di-tert-butylhydroquinone (DBHQ)': '2,5-di-tert-butylbenzene-1,4-diol',
 'Benzophenone (BP-5 and BP-9)': 'Diphenylmethanone',
 'Tanshinone IIA': None,
 '1-ethyl-3-methylimidazoliumiodide': '1-ethyl-3-methylimidazolium iodide',
 '1,3-dimethyl-3,4,5,6-tetrahydro-2(1H)-pyrimidinone (DMPU)': '1,3-dimethyl-2-imidazolidinone',
 'CoBr₂': 'Cobalt(II) bromide',
 'FeCO3, Fe2O3, Fe3O4, α-FeOOH, γ-FeOOH': None,
 'Imidazoanthraquinone derivative (AQ)': None,
 "4,4'-oxybisbenzoic acid (OBBA)": "4,4'-oxybisbenzoic acid",
 '[EMIM] + [PF6]^-': '1-ethyl-3-methylimidazolium hexafluorophosphate',
 "N, N'-Dimethyl-1,2-ethanediamine (DMEDA)": "N,N'-dimethylethane-1,2-diamine",
 "N,N'-di(4-pyridyl)-1,4,5,8-naphthalenetetracarboxdiimide (DPNDI)": "N,N'-bis(4-pyridyl)-1,4,5,8-naphthalene tetracarboxdiimide",
 'Benzylamine, phenylethylamine bromide, guanidinium bromide': None,
 'Europium (Eu^3+/Eu^2+) and sulfonate anthraquinone-based molecules, and thiol/disulfide couples': None,
 'OAI (octylammonium iod

In [ ]:
for key in extracted_passivators:
    passivator_conversions[key] = extracted_passivators[key]

In [97]:
passivator_conversions

{'natural alkene lycopene': '2-(6,6,6-Trimethyl-1-cyclohexen-1-yl)-bicyclo[4.2.0]octa-1,3,5-triene',
 'F-15': None,
 '4-ethenyl-2,6-dimethoxyphenol (canolol, CNL)': '4-ethenyl-2,6-dimethoxyphenol',
 'PbS quantum dots': None,
 '1-methyl-3-propylimidazolium iodide (1-MPII)': '1-methyl-3-propylimidazolium iodide',
 'Cu2(Tu)Br2': None,
 'Gly-E': None,
 'MPA-TB-CA': None,
 'K-NAA': None,
 '4-Acetamidobenzoic acid': '4-acetamidobenzoic acid',
 'MoO_x': None,
 'morpholinium iodide (MPI)': 'morpholinium iodide',
 '2D perovskite nanoplatelets-PMMA composite': None,
 'N,N-methylenebisacrylamide (MBA)': 'N,N-methylenebisacrylamide',
 '4-methanesulfonyl-benzamidine hydrochloride (MSBH)': '4-methanesulfonyl-benzamidine hydrochloride',
 '3-(2-aminoethyl)pyridine (3-PyEA)': '3-(2-aminoethyl)pyridine',
 'MgO': 'magnesium oxide',
 'Nylon 11 (N11)': None,
 'GQD-doped ZnO': None,
 'PMMA': 'poly(methyl methacrylate)',
 'usnic acid': 'usnic acid',
 'AMSA': None,
 'TPCA': None,
 'FAMAPbI-BrCl': None,
 'Phen

In [98]:
with open('data/passivator_conversions.json', 'w') as f:
    json.dump(passivator_conversions, f)

In [50]:
results = {}
for key in passivator_conversions:
    passivator = passivator_conversions[key]
    if passivator is not None:
        print(passivator)
        base_url = "https://opsin.ch.cam.ac.uk/opsin/"
        smiles_url = base_url + passivator + ".smi"
        r = requests.get(smiles_url)
        result = r.text if r.status_code == 200 else None
        results[passivator] = result

2-(6,6,6-Trimethyl-1-cyclohexen-1-yl)-bicyclo[4.2.0]octa-1,3,5-triene
4-ethenyl-2,6-dimethoxyphenol
1-methyl-3-propylimidazolium iodide
4-acetamidobenzoic acid
morpholinium iodide
N,N-methylenebisacrylamide
4-methanesulfonyl-benzamidine hydrochloride
3-(2-aminoethyl)pyridine
magnesium oxide
poly(methyl methacrylate)
usnic acid
trimethylphenylammonium
formamidine acetate
zinc nitride
2,3-dimercaptopropanol
pyrrolidinium thiocyanate
thulium oxide
3,4,5-trifluoroaniline iodide
ethylenediamine
formamidinium
iron-doped zinc oxide
dodecylammonium formate
2,8-diiododibenzothiophene
methylammonium iodide
polyethylene oxide
copper(II)
cesium fluoride
methylammonium lead iodide
octylammonium
copper(I) oxide
methanol
phenylethylammonium iodide
sodium dodecyl sulfate
diethyl ether
phenethylammonium iodide
sodium chloride
sodium dodecyl sulfate
choline acetate
1-ethyl-3-methylimidazolium acetate
polyvinylidene fluoride
N,N′-dimethyl-1,3-propanediammonium dichloride
anethole
2,4-diamino-6-fluoropyri

KeyboardInterrupt: 

In [51]:
results

{'2-(6,6,6-Trimethyl-1-cyclohexen-1-yl)-bicyclo[4.2.0]octa-1,3,5-triene': None,
 '4-ethenyl-2,6-dimethoxyphenol': 'C(=C)C1=CC(=C(C(=C1)OC)O)OC',
 '1-methyl-3-propylimidazolium iodide': '[I-].CN1C=[N+](C=C1)CCC',
 '4-acetamidobenzoic acid': 'C(C)(=O)NC1=CC=C(C(=O)O)C=C1',
 'morpholinium iodide': '[I-].[NH2+]1CCOCC1',
 'N,N-methylenebisacrylamide': None,
 '4-methanesulfonyl-benzamidine hydrochloride': 'Cl.CS(=O)(=O)C1=CC=C(C(=N)N)C=C1',
 '3-(2-aminoethyl)pyridine': 'NCCC=1C=NC=CC1',
 'magnesium oxide': '[O-2].[Mg+2]',
 'poly(methyl methacrylate)': None,
 'usnic acid': None,
 'trimethylphenylammonium': 'C[N+](C1=CC=CC=C1)(C)C',
 'formamidine acetate': 'C(C)(=O)O.C(=N)N',
 'zinc nitride': None,
 '2,3-dimercaptopropanol': 'SC(CO)CS',
 'pyrrolidinium thiocyanate': '[S-]C#N.[NH2+]1CCCC1',
 'thulium oxide': '[O-2].[Tm+3].[O-2].[O-2].[Tm+3]',
 '3,4,5-trifluoroaniline iodide': '[I-].FC=1C=C(N)C=C(C1F)F',
 'ethylenediamine': 'C(CN)N',
 'formamidinium': 'C(=[NH2+])N',
 'iron-doped zinc oxide': Non